## neural ODE with exogenous inputs applied on CED and CCT benchmarks

* includes exogenous inputs
* includes the introduced normalization term

In [1]:
import enum
import os
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
import matplotlib

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.auto import tqdm

from torchdiffeq import odeint #includes exogenous inputs

class LatentODEfunc(nn.Module):

    def __init__(self, latent_dim=4, nhidden=20, udim=None):
        super(LatentODEfunc, self).__init__()
        self.elu = nn.ELU(inplace=True)
        udim_val = 0 if udim is None else udim
        self.fc1 = nn.Linear(latent_dim + udim_val, nhidden)
        self.fc2 = nn.Linear(nhidden, nhidden)
        self.fc3 = nn.Linear(nhidden, latent_dim)
        self.nfe = 0

    def forward(self, t, x, u=None):
        self.nfe += 1
        if u is not None: #append the input to the state
            x = torch.cat([x,u[:,None] if u.ndim==1 else u],dim=1)
        out = self.fc1(x)
        out = self.elu(out)
        out = self.fc2(out)
        out = self.elu(out)
        out = self.fc3(out)
        return out


class RecognitionRNN(nn.Module):

    def __init__(self, latent_dim=4, obs_dim=2, nhidden=25, nbatch=1):
        super(RecognitionRNN, self).__init__()
        self.nhidden = nhidden
        self.nbatch = nbatch
        self.i2h = nn.Linear(obs_dim + nhidden, nhidden)
        self.h2o = nn.Linear(nhidden, latent_dim * 2)

    def forward(self, x, h):
        combined = torch.cat((x, h), dim=1)
        h = torch.tanh(self.i2h(combined))
        out = self.h2o(h)
        return out, h

    def initHidden(self):
        return torch.zeros(self.nbatch, self.nhidden)


class Decoder(nn.Module):

    def __init__(self, latent_dim=4, obs_dim=2, nhidden=20):
        super(Decoder, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(latent_dim, nhidden)
        self.fc2 = nn.Linear(nhidden, obs_dim)

    def forward(self, z):
        out = self.fc1(z)
        out = self.relu(out)
        out = self.fc2(out)
        return out


class RunningAverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val


def log_normal_pdf(x, mean, logvar):
    const = torch.from_numpy(np.array([2. * np.pi])).float().to(x.device)
    const = torch.log(const)
    return -.5 * (const + logvar + (x - mean) ** 2. / torch.exp(logvar))


def normal_kl(mu1, lv1, mu2, lv2):
    v1 = torch.exp(lv1)
    v2 = torch.exp(lv2)
    lstd1 = lv1 / 2.
    lstd2 = lv2 / 2.

    kl = lstd2 - lstd1 + ((v1 + (mu1 - mu2) ** 2.) / (2. * v2)) - .5
    return kl

def get_train_val_test(dataset):
    import deepSI
    from deepSI import System_data_list
    if dataset=='CED':
        data_full = deepSI.datasets.CED()
        train = System_data_list([data_i[:300] for data_i in data_full])
        test = System_data_list([data_i[300:] for data_i in data_full])
        val = System_data_list([t[:100] for t in test])
    elif dataset=='CCT':
        train, test = deepSI.datasets.Cascaded_Tanks()
        val, test = test[:len(test)//2], test
    return train, val, test

def get_torch_data(dataset, device=torch.device('cpu')):
    train, val, test = get_train_val_test(dataset)
    from deepSI.system_data import System_data_norm
    norm = System_data_norm() #normalization
    norm.fit(train)
    train, val, test = [norm.transform(t) for t in [train, val, test]]

    if dataset=='CCT':
        samp_trajs = train.y[None,:,None]
        u_samp_trajs = train.u[None,:,None]
        orig_trajs = test.y[None,:,None]
        u_orig_trajs = test.u[None,:,None]
        val_trajs = val.y[None,:,None]
        u_val_trajs = val.u[None,:,None]

        sample_time = 4. #seconds
        samp_ts = np.arange(len(train.y))*sample_time

    elif dataset=='CED':
        samp_trajs = np.array([ti.y[:,None] for ti in train])
        u_samp_trajs =  np.array([ti.u for ti in train])
        orig_trajs = np.array([ti.y[:,None] for ti in test])
        u_orig_trajs = np.array([ti.u for ti in test])
        val_trajs = np.array([ti.y[:,None] for ti in val])
        u_val_trajs = np.array([ti.u[:,None] for ti in val])

        sample_time = 1/50 #seconds
        samp_ts = np.arange(len(train[0]))*sample_time

    orig_trajs = torch.from_numpy(orig_trajs).float().to(device) #samples, time
    u_orig_trajs = torch.from_numpy(u_orig_trajs).float().to(device)
    samp_trajs = torch.from_numpy(samp_trajs).float().to(device)
    u_samp_trajs = torch.from_numpy(u_samp_trajs).float().to(device)
    samp_ts = torch.from_numpy(samp_ts).float().to(device)
    u_val_trajs = torch.from_numpy(u_val_trajs).float().to(device)
    val_trajs = torch.from_numpy(val_trajs).float().to(device)
    
    return orig_trajs, u_orig_trajs, samp_trajs, u_samp_trajs, samp_ts, \
            u_val_trajs, val_trajs, sample_time, norm

In [16]:
dataset='CED'
method = 'rk4' #exogenous inputs only implemented for Euler, RK4 and midpoint.
device = torch.device('cpu')
train_dir = './models-neural-ode/'
niters = 20000

#given by latent_ODE:
nhidden = 20
rnn_nhidden = 25
obs_dim = 1
noise_std = 0.1

orig_trajs, u_orig_trajs, samp_trajs, u_samp_trajs, samp_ts, u_val_trajs, val_trajs, sample_time, norm = \
        get_torch_data(dataset, device=device)

tau_1 = True

if dataset=='CCT':
    dttau = 0.032
    latent_dim = 2
    
elif dataset=='CED':
    dttau = 0.12
    latent_dim = 3
    
tau = 1 if tau_1 else sample_time/dttau
    
samp_ts /= tau #dt /dt * dttau = dttau

has_val = True

In [ ]:
for I in range(1,100):
    
    # create model
    print(f'I={I}')
    udim = 1 if u_orig_trajs.ndim==2 else u_orig_trajs.shape[-1]
    func = LatentODEfunc(latent_dim, nhidden, udim=udim).to(device)
    rec = RecognitionRNN(latent_dim, obs_dim, rnn_nhidden, samp_trajs.shape[0]).to(device)
    dec = Decoder(latent_dim, obs_dim, nhidden).to(device)
    params = (list(func.parameters()) + list(dec.parameters()) + list(rec.parameters()))
    optimizer = optim.Adam(params, lr=0.01)
    loss_meter = RunningAverageMeter()

    try: #for KeyboardInterrupt stopping of iterations
        best_val = float('inf')
        for itr in tqdm(range(1, niters + 1)):
            optimizer.zero_grad()
            # backward in time to infer q(z_0)
            h = rec.initHidden().to(device)
            for t in reversed(range(samp_trajs.size(1))):
                obs = samp_trajs[:, t, :]
                out, h = rec.forward(obs, h)
            qz0_mean, qz0_logvar = out[:, :latent_dim], out[:, latent_dim:]
            epsilon = torch.randn(qz0_mean.size()).to(device)
            z0 = epsilon * torch.exp(.5 * qz0_logvar) + qz0_mean

            # forward in time and solve ode for reconstructions
            pred_z = odeint(func, z0, samp_ts, u=u_samp_trajs, method=method).permute(1, 0, 2)
            pred_x = dec(pred_z)

            # compute loss
            noise_std_ = torch.zeros(pred_x.size()).to(device) + noise_std
            noise_logvar = 2. * torch.log(noise_std_).to(device)
            logpx = log_normal_pdf(
                samp_trajs, pred_x, noise_logvar).sum(-1).sum(-1)
            pz0_mean = pz0_logvar = torch.zeros(z0.size()).to(device)
            analytic_kl = normal_kl(qz0_mean, qz0_logvar,
                                    pz0_mean, pz0_logvar).sum(-1)
            loss = torch.mean(-logpx + analytic_kl, dim=0)
            loss.backward()
            optimizer.step()
            loss_meter.update(loss.item())


            if has_val and itr%10==0:

                h = rec.initHidden().to(device)
                for t in reversed(range(val_trajs.size(1))):
                    obs = val_trajs[:, t, :]
                    out, h = rec.forward(obs, h)
                qz0_mean, qz0_logvar = out[:, :latent_dim], out[:, latent_dim:]
                epsilon = torch.randn(qz0_mean.size()).to(device)
                z0 = epsilon * torch.exp(.5 * qz0_logvar) + qz0_mean

                # forward in time and solve ode for reconstructions
                pred_z = odeint(func, z0, samp_ts[:u_val_trajs.shape[1]], u=u_val_trajs, method=method).permute(1, 0, 2)
                pred_x = dec(pred_z)
                val_trajs_p = pred_x.detach()
                RMS = torch.mean((val_trajs_p - val_trajs)**2).item()**0.5
                print('Iter: {}, running avg elbo: {:.4f} rms val {:.4f}'.format(itr, -loss_meter.avg, RMS))
                if RMS<best_val:
                    best_val = RMS
                    if train_dir is not None:
                        if tau_1:
                            ckpt_path = os.path.join(train_dir, f'ckpt-best-{dataset}-{I}-tau_1.pth')
                        else:
                            ckpt_path = os.path.join(train_dir, f'ckpt-best-{dataset}-{I}.pth')
                        torch.save({
                            'func':func,
                            'rec':rec,
                            'dec':dec,
                            'func_state_dict': func.state_dict(),
                            'rec_state_dict': rec.state_dict(),
                            'dec_state_dict': dec.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'orig_trajs': orig_trajs,
                            'samp_trajs': samp_trajs,
                            'samp_ts': samp_ts,
                        }, ckpt_path)
                        print('Stored ckpt at {} {}'.format(ckpt_path, RMS))

    except KeyboardInterrupt:
        print(f'stopped by keyboard I={I}')
        break
    if train_dir is not None:
        if tau_1:
            ckpt_path = os.path.join(train_dir, f'ckpt-last-{dataset}-{I}-tau_1.pth')
        else:
            ckpt_path = os.path.join(train_dir, f'ckpt-last-{dataset}-{I}.pth')
        torch.save({
            'func':func,
            'rec':rec,
            'dec':dec,
            'func_state_dict': func.state_dict(),
            'rec_state_dict': rec.state_dict(),
            'dec_state_dict': dec.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'orig_trajs': orig_trajs,
            'samp_trajs': samp_trajs,
            'samp_ts': samp_ts,
        }, ckpt_path)
        print('Stored ckpt at {}'.format(ckpt_path))
    print('Training complete after {} iters.'.format(itr))

I=1


  0%|          | 0/20000 [00:00<?, ?it/s]

Iter: 10, running avg elbo: -14897.2547 rms val 0.7117
Stored ckpt at ./models-neural-ode/ckpt-best-CED-1-tau_1.pth 0.7117038829938203
Iter: 20, running avg elbo: -14811.3614 rms val 0.7327
Iter: 30, running avg elbo: -14565.5306 rms val 0.6480
Stored ckpt at ./models-neural-ode/ckpt-best-CED-1-tau_1.pth 0.64801894318007
Iter: 40, running avg elbo: -14013.2320 rms val 0.6727
Iter: 50, running avg elbo: -13053.4073 rms val 0.7055
Iter: 60, running avg elbo: -12018.5002 rms val 0.6223
Stored ckpt at ./models-neural-ode/ckpt-best-CED-1-tau_1.pth 0.622263736087401
Iter: 70, running avg elbo: -11010.3678 rms val 0.5700
Stored ckpt at ./models-neural-ode/ckpt-best-CED-1-tau_1.pth 0.5699506160190284
Iter: 80, running avg elbo: -10106.3659 rms val 0.6475
Iter: 90, running avg elbo: -9261.1354 rms val 0.5167
Stored ckpt at ./models-neural-ode/ckpt-best-CED-1-tau_1.pth 0.5167248584165937
Iter: 100, running avg elbo: -8492.5021 rms val 0.5109
Stored ckpt at ./models-neural-ode/ckpt-best-CED-1-tau

Iter: 1370, running avg elbo: -371.7231 rms val 0.8017
Iter: 1380, running avg elbo: -371.8167 rms val 0.7460
Iter: 1390, running avg elbo: -372.8577 rms val 0.7596
Iter: 1400, running avg elbo: -370.9001 rms val 0.8143
Iter: 1410, running avg elbo: -368.6292 rms val 0.7734
Iter: 1420, running avg elbo: -369.8883 rms val 0.7659
Iter: 1430, running avg elbo: -370.1893 rms val 0.7651
Iter: 1440, running avg elbo: -371.4432 rms val 0.7824
Iter: 1450, running avg elbo: -369.2586 rms val 0.7636
Iter: 1460, running avg elbo: -365.2423 rms val 0.7742
Iter: 1470, running avg elbo: -361.5694 rms val 0.7949
Iter: 1480, running avg elbo: -357.8097 rms val 0.8298
Iter: 1490, running avg elbo: -354.4432 rms val 0.7819
Iter: 1500, running avg elbo: -350.5677 rms val 0.8048
Iter: 1510, running avg elbo: -349.0944 rms val 0.8339
Iter: 1520, running avg elbo: -347.1163 rms val 0.7969
Iter: 1530, running avg elbo: -344.1029 rms val 0.8144
Iter: 1540, running avg elbo: -340.7916 rms val 0.8268
Iter: 1550

Iter: 2860, running avg elbo: -247.9394 rms val 0.7597
Iter: 2870, running avg elbo: -249.2021 rms val 0.7438
Iter: 2880, running avg elbo: -246.6929 rms val 0.7450
Iter: 2890, running avg elbo: -244.8311 rms val 0.7355
Iter: 2900, running avg elbo: -243.5591 rms val 0.7275
Iter: 2910, running avg elbo: -241.6738 rms val 0.7400
Iter: 2920, running avg elbo: -238.5905 rms val 0.7438
Iter: 2930, running avg elbo: -236.4360 rms val 0.7584
Iter: 2940, running avg elbo: -234.7672 rms val 0.7282
Iter: 2950, running avg elbo: -235.0466 rms val 0.7695
Iter: 2960, running avg elbo: -234.7697 rms val 0.7222
Iter: 2970, running avg elbo: -233.2246 rms val 0.7489
Iter: 2980, running avg elbo: -230.0993 rms val 0.7390
Iter: 2990, running avg elbo: -227.9853 rms val 0.7437
Iter: 3000, running avg elbo: -227.0117 rms val 0.7506
Iter: 3010, running avg elbo: -226.0145 rms val 0.7452
Iter: 3020, running avg elbo: -223.9228 rms val 0.7553
Iter: 3030, running avg elbo: -220.8850 rms val 0.7449
Iter: 3040

...

  0%|          | 0/20000 [00:00<?, ?it/s]

Iter: 10, running avg elbo: -14888.2810 rms val 0.7817
Stored ckpt at ./models-neural-ode/ckpt-best-CED-12-tau_1.pth 0.781749451251885
Iter: 20, running avg elbo: -14887.4495 rms val 0.7068
Stored ckpt at ./models-neural-ode/ckpt-best-CED-12-tau_1.pth 0.7067831457682472
Iter: 30, running avg elbo: -14845.7230 rms val 0.6575
Stored ckpt at ./models-neural-ode/ckpt-best-CED-12-tau_1.pth 0.6575238262689487
Iter: 40, running avg elbo: -14734.8428 rms val 0.6377
Stored ckpt at ./models-neural-ode/ckpt-best-CED-12-tau_1.pth 0.6377189363131744
Iter: 50, running avg elbo: -14329.2659 rms val 0.6412
Iter: 60, running avg elbo: -13451.3481 rms val 0.9406
Iter: 70, running avg elbo: -12543.5456 rms val 0.8313
Iter: 80, running avg elbo: -11696.5037 rms val 0.7623
Iter: 90, running avg elbo: -10874.0393 rms val 0.6568
Iter: 100, running avg elbo: -10087.1299 rms val 0.6753
Iter: 110, running avg elbo: -9340.8902 rms val 0.6465
Iter: 120, running avg elbo: -8635.7435 rms val 0.5878
Stored ckpt at .

Iter: 1410, running avg elbo: -622.1854 rms val 1.1693
Iter: 1420, running avg elbo: -617.7650 rms val 1.2206
Iter: 1430, running avg elbo: -613.9667 rms val 1.2179
Iter: 1440, running avg elbo: -609.9911 rms val 1.2966
Iter: 1450, running avg elbo: -610.6838 rms val 1.2868
Iter: 1460, running avg elbo: -611.0634 rms val 1.0903
Iter: 1470, running avg elbo: -606.8920 rms val 1.0262
Iter: 1480, running avg elbo: -601.9927 rms val 1.0336
Iter: 1490, running avg elbo: -597.7941 rms val 1.1503
Iter: 1500, running avg elbo: -594.3119 rms val 1.1180
Iter: 1510, running avg elbo: -590.7747 rms val 1.0613
Iter: 1520, running avg elbo: -586.8618 rms val 0.9800
Iter: 1530, running avg elbo: -587.5007 rms val 1.0005
Iter: 1540, running avg elbo: -587.0397 rms val 0.9579
Iter: 1550, running avg elbo: -588.3694 rms val 0.9267
Iter: 1560, running avg elbo: -589.2011 rms val 0.9718
Iter: 1570, running avg elbo: -588.4020 rms val 1.0653
Iter: 1580, running avg elbo: -589.0889 rms val 1.0149
Iter: 1590

Iter: 2900, running avg elbo: -268.2265 rms val 1.4508
Iter: 2910, running avg elbo: -265.0801 rms val 1.3774
Iter: 2920, running avg elbo: -266.9553 rms val 1.3923
Iter: 2930, running avg elbo: -269.5906 rms val 1.3902
Iter: 2940, running avg elbo: -267.1845 rms val 1.3809
Iter: 2950, running avg elbo: -261.9277 rms val 1.3815
Iter: 2960, running avg elbo: -254.7704 rms val 1.3916
Iter: 2970, running avg elbo: -257.8214 rms val 1.3685
Iter: 2980, running avg elbo: -276.9643 rms val 1.3517
Iter: 2990, running avg elbo: -287.7208 rms val 1.4062
Iter: 3000, running avg elbo: -284.8415 rms val 1.3979
Iter: 3010, running avg elbo: -276.7687 rms val 1.3897
Iter: 3020, running avg elbo: -267.6966 rms val 1.3698
Iter: 3030, running avg elbo: -258.6680 rms val 1.4202
Iter: 3040, running avg elbo: -251.8032 rms val 1.3838
Iter: 3050, running avg elbo: -250.2774 rms val 1.3632
Iter: 3060, running avg elbo: -247.1440 rms val 1.3572
Iter: 3070, running avg elbo: -242.0413 rms val 1.3605
Iter: 3080

Iter: 4410, running avg elbo: -25.5657 rms val 1.2491
Iter: 4420, running avg elbo: -24.3290 rms val 1.2609
Iter: 4430, running avg elbo: -22.9070 rms val 1.2598
Iter: 4440, running avg elbo: -20.8273 rms val 1.2759
Iter: 4450, running avg elbo: -17.2413 rms val 1.2828
Iter: 4460, running avg elbo: -13.3247 rms val 1.3043
Iter: 4470, running avg elbo: -10.9930 rms val 1.2669
Iter: 4480, running avg elbo: -14.0732 rms val 1.2185
Iter: 4490, running avg elbo: -26.7686 rms val 1.4001
Iter: 4500, running avg elbo: -32.8175 rms val 1.3220
Iter: 4510, running avg elbo: -31.8385 rms val 1.2937
Iter: 4520, running avg elbo: -29.0681 rms val 1.3169
Iter: 4530, running avg elbo: -24.2590 rms val 1.2655
Iter: 4540, running avg elbo: -18.7318 rms val 1.2922
Iter: 4550, running avg elbo: -14.0051 rms val 1.2662
Iter: 4560, running avg elbo: -14.0711 rms val 1.2934
Iter: 4570, running avg elbo: -12.9317 rms val 1.3458
Iter: 4580, running avg elbo: -12.8259 rms val 1.2414
Iter: 4590, running avg elbo